In [1]:
# Imports
import os
import numpy as np
import SimpleITK as sitk
import multiprocessing
from glob import glob
import pandas as pd
import time
import warnings

from utils.utilities import check_if_exist
from utils.load_data import load_data
from utils.voi_extraction import run_voi_extraction
from utils.postprocessing import postprocessing_native, export2dicomRT
from utils.dm_computation import dm_computation

# Set up paths
ddbb      = 'PRUEBAS'
mode      = 'dicom' #'nifti'
modality  = 'CT'
model     = 'FR_model' #'Mixed_model'
use_manual_OARs = True # False
data_path = os.getcwd()+'/Input/'+ddbb
check_if_exist(data_path, create=False) # Check that the path exists
out_path  = os.path.join(os.getcwd(), 'Output', ddbb, modality)
check_if_exist(out_path)

/home/igt/Projects/PerPlanRT/GITHUB/Input/PRUEBAS .. exists
/home/igt/Projects/PerPlanRT/GITHUB/Output/PRUEBAS/CT .. exists


In [2]:
start_ini = time.time()
# 1. Load patients (DICOM or NIfTI images)
print("-------------------------------------------------------------------------------------------------------")
print("1. LOADING ORIGINAL IMAGES...")
print("-------------------------------------------------------------------------------------------------------")
# CT and MR data have to be loaded separately, to indicate if they are in DICOM or NIfTI format
load_data(data_path, out_path, mode, modality=modality)

-------------------------------------------------------------------------------------------------------
1. LOADING ORIGINAL IMAGES...
-------------------------------------------------------------------------------------------------------
Loading data...

Processing case:  0001
----- CT Image -----
/home/igt/Projects/PerPlanRT/GITHUB/Input/PRUEBAS/0001/CT
(-346.0, -261.0, -260.0)
(1.359375, 1.359375, 1.0)
(512, 512, 366)
Saving... /home/igt/Projects/PerPlanRT/GITHUB/Output/PRUEBAS/CT/CTs/PRUEBAS_0001_0000.nii.gz
----- Segmentation -----
['PTV', 'Prostate-CTV', 'Voptim', 'Rectum', 'Bladder', 'Bowel', 'Penile Bulb', 'Left Femoral Head', 'Right Femoral Head', 'BrasDensite0', 'Skin', 'teteFdt+G', 'prostate', 'pv', 'pr', 'Overlapspv', 'Overlapspr', 'Overlapsrectu', 'Overlapsvesiie', 'sigmoide']
No seminal vesicles in manual OARs segmentations
['Rectum', 'Bladder', 'prostate']
Labels:  [0 1 2 3]
Saving... /home/igt/Projects/PerPlanRT/GITHUB/Output/PRUEBAS/CT/GTs/0001/labelMap.nii.gz

Processi

In [3]:
# 2. VOI Extraction: Localization Network + Crop using the centroid of the coarse segmentation. Check the result to ensure that
# appropriate VOI has been created. Sometimes some images are not well predicted and it's necessary to modify this VOI manually
# to ensure that the OARs and urethra segmentations are accurate.
print("-------------------------------------------------------------------------------------------------------")
print("2. VOI EXTRACTION...")
print("-------------------------------------------------------------------------------------------------------")
checkpoint_path = os.getcwd()+'/networks/LocalizationNet/LocalizationNet_weights.best.hdf5'
dir_ddbb     = os.path.join(out_path, modality+'s')
if use_manual_OARs: 
    check_if_exist(os.path.join(out_path, 'mVOIs', 'imagesTs')) # Path to save generated VOIs
    check_if_exist(os.path.join(out_path, 'OARs', 'manual'))
else: check_if_exist(os.path.join(out_path, 'VOIs', 'imagesTs')) # Path to save generated VOIs

p = multiprocessing.Process(target=run_voi_extraction, args=(checkpoint_path, out_path, dir_ddbb, use_manual_OARs))
p.start()
p.join()

-------------------------------------------------------------------------------------------------------
2. VOI EXTRACTION...
-------------------------------------------------------------------------------------------------------
/home/igt/Projects/PerPlanRT/GITHUB/Output/PRUEBAS/CT/mVOIs/imagesTs .. exists
/home/igt/Projects/PerPlanRT/GITHUB/Output/PRUEBAS/CT/OARs/manual .. exists

Processing case:  0001
x: 362.93919398813205 y: 343.7555616651774 z: 147.07819552317457
x_offset 250 : 474 y_offset 231 : 455 z_offset 35 : 259
(366, 696, 696)
Saving... /home/igt/Projects/PerPlanRT/GITHUB/Output/PRUEBAS/CT/mVOIs/imagesTs/PRUEBAS_0001_0000.nii.gz
x_offset 250 : 474 y_offset 231 : 455 z_offset 35 : 259
(366, 696, 696)


In [4]:
# 3. OARs Segmentation: Fine Segmentation Network
print("-------------------------------------------------------------------------------------------------------")
print("3. OARs SEGMENTATION (OUTPUT 1)...")
print("-------------------------------------------------------------------------------------------------------")
# Environmnet Variables
main_dir  = os.path.join(os.getcwd(), 'networks/SegmentationNet/nnUNet')
# Can be set in the the .bashrc file or exported in the terminal:
os.environ['nnUNet_raw_data_base'] = os.path.join(main_dir,'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed']  = os.path.join(main_dir,'nnUNet_preprocessed')
os.environ['RESULTS_FOLDER']       = os.path.join(main_dir,'nnUNet_trained_models')
os.environ['MKL_SERVICE_FORCE_INTEL'] = "1"

if use_manual_OARs:
    model = 'manual'
    path_imgs    = os.path.join(out_path, 'mVOIs', 'imagesTs')
    dir_ddbb_OARs = os.path.join(out_path, 'OARs', model)

else:
    path_imgs    = os.path.join(out_path, 'VOIs', 'imagesTs')
    dir_ddbb_OARs = os.path.join(out_path, 'OARs', model)

    # Predict OAR segmentations (rectum, bladder, prostate, seminal vesicles)
    if model=='FR_model':
        !nnUNet_predict -i {path_imgs} -o {dir_ddbb_OARs} -t 112 -tr nnUNetTrainerV2 -ctr nnUNetTrainerV2CascadeFullRes -m 3d_fullres -p nnUNetPlansv2.1 --disable_tta

    if model=='Mixed_model':
        !nnUNet_predict -i {path_imgs} -o {dir_ddbb_OARs} -t 113 -tr nnUNetTrainerV2 -ctr nnUNetTrainerV2CascadeFullRes -m 3d_fullres -p nnUNetPlansv2.1 --disable_tta

    for i, file_OARs in enumerate(sorted(glob(dir_ddbb_OARs + r'/*.nii.gz'))):
        idx = file_OARs.split('/')[-1].split('_')[-1].split('.')[0]
        print(file_OARs)
        out_OARs  = sitk.ReadImage(file_OARs)
        out_VOI   = sitk.ReadImage(os.path.join(path_imgs, ddbb+'_'+idx+'_0000.nii.gz'))

        # 3.1 Post-processing to Native space
        print("3.1 OARs SEGMENTATION POST-PROCESSING TO NATIVE SPACE -------------------------------------------------")
        postprocessing_native(out_OARs, out_path, str(idx),'OARs', ddbb, modality)
        postprocessing_native(out_VOI, out_path, str(idx),'VOI', ddbb, modality)

        # 3.2 Exportation to Dicom-RT
        print("3.2 OARs SEGMENTATION EXPORTATION TO DICOM-RT ---------------------------------------------------------")
        img_file  = os.path.join(out_path, 'Native', str(idx), ddbb+'_'+str(idx)+'_VOI.nii.gz')
        seg_file = os.path.join(out_path, 'Native', str(idx), ddbb+'_'+str(idx)+'_OARs.nii.gz')
        save_path = os.path.join(out_path, 'Native', 'DICOM', str(idx))
        # For the moment, we can only translate CTs and their segmentations to DICOM; not MR images
        if modality=='CT':
            export2dicomRT(img_file, seg_file, save_path, 'OARs_DLUS', modality)

-------------------------------------------------------------------------------------------------------
3. OARs SEGMENTATION (OUTPUT 1)...
-------------------------------------------------------------------------------------------------------


In [4]:
# 4. DISTANCE MAP COMPUTATION
print("-------------------------------------------------------------------------------------------------------")
print("4. DISTANCE MAP COMPUTATION...")
print("-------------------------------------------------------------------------------------------------------")
if use_manual_OARs: check_if_exist(os.path.join(out_path, 'mDistanceMaps', 'imagesTs'))
else: check_if_exist(os.path.join(out_path, 'DistanceMaps', 'imagesTs'))
# Bladder & Prostate Segmentation and DM Computation
dm_computation(dir_ddbb, out_path, model, use_manual_OARs)

-------------------------------------------------------------------------------------------------------
4. DISTANCE MAP COMPUTATION...
-------------------------------------------------------------------------------------------------------
/home/igt/Projects/PerPlanRT/GITHUB/Output/PRUEBAS/CT/mDistanceMaps/imagesTs .. exists

Processing case:  0001
--- Processing DM total : 18.570481777191162 seconds ---
Saving... /home/igt/Projects/PerPlanRT/GITHUB/Output/PRUEBAS/CT/mDistanceMaps/imagesTs/PRUEBAS_0001_0000.nii.gz


In [5]:
# 5. URETHRA SEGMENTATION
print("-------------------------------------------------------------------------------------------------------")
print("5. URETHRA SEGMENTATION")
print("-------------------------------------------------------------------------------------------------------")

if use_manual_OARs:
    path_dm = os.path.join(out_path, 'mDistanceMaps', 'imagesTs')
    dir_ddbb_uretra = os.path.join(out_path, 'Urethra', 'manual')
else:
    path_dm = os.path.join(out_path, 'DistanceMaps', 'imagesTs')
    dir_ddbb_uretra = os.path.join(out_path, 'Urethra', 'DLUS')

!nnUNet_predict -i {path_dm} -o {dir_ddbb_uretra} -t 108 -tr nnUNetTrainerV2 -ctr nnUNetTrainerV2CascadeFullRes -m 3d_fullres -p nnUNetPlansv2.1 --disable_tta

-------------------------------------------------------------------------------------------------------
5. URETHRA SEGMENTATION
-------------------------------------------------------------------------------------------------------


Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

nnUNet_raw_data_base is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for prepro

In [7]:
for i, file_out_urethra in enumerate(sorted(glob(dir_ddbb_uretra + r'/*.nii.gz'))):
    idx = file_out_urethra.split('/')[-1].split('_')[-1].split('.')[0]
    print(file_out_urethra)
    out_urethra = sitk.ReadImage(file_out_urethra)
    
    # 5.1 Post-processing to Native space
    print("5.1 URETHRA SEGMENTATION POST-PROCESSING TO NATIVE SPACE ----------------------------------------------")
    if use_manual_OARs:    postprocessing_native(out_urethra, out_path, str(idx),'m_urethra', ddbb, modality)
    else: postprocessing_native(out_urethra, out_path, str(idx),'urethra', ddbb, modality)
    
    # 5.2 Exportation to Dicom-RT
    print("5.2 URETHRA SEGMENTATION EXPORTATION TO DICOM-RT ------------------------------------------------------")
    img_file  = os.path.join(out_path, 'Native', str(idx), ddbb+'_'+str(idx)+'_VOI.nii.gz')
    seg_file = os.path.join(out_path, 'Native', str(idx), ddbb+'_'+str(idx)+'_urethra.nii.gz')
    save_path = os.path.join(out_path, 'Native', 'DICOM', str(idx))
    # For the moment, we can only translate CTs and their segmentations to DICOM; not MR images
    if modality=='CT':
        export2dicomRT(img_file, seg_file, save_path, 'urethra_DLUS', modality)

print("--- Total Execution Time: %s seconds ---" % (time.time() - start_ini))